In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import astropy.units as u
%matplotlib inline
%config InlineBackend.figure_format = "retina"
from matplotlib import rcParams
rcParams['savefig.dpi'] = 550
rcParams['font.size'] = 20
plt.rc('font', family='serif')

import lsdb
from lsdb import lsdb_client
client = lsdb_client(dask_on_ray=False, num_workers=12)

In [2]:
client.client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:43220/status,
Dashboard: http://127.0.0.1:43220/status,Workers: 12
Total threads: 12,Total memory: 0.98 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41765,Workers: 12
Dashboard: http://127.0.0.1:43220/status,Total threads: 12
Started: Just now,Total memory: 0.98 TiB
Comm: tcp://127.0.0.1:36267,Total threads: 1
Dashboard: http://127.0.0.1:40468/status,Memory: 83.97 GiB
Nanny: tcp://127.0.0.1:34103,


In [3]:
#Load ZTF, Gaia, and ZTF sources hipscats
gaia = lsdb.read_hipscat("/data3/epyc/projects3/ivoa_demo/gaia/catalog", 
                        columns=['ra', 'dec', 'bp_rp', 
                                'parallax', 'parallax_over_error', 
                                'phot_g_mean_mag', 
                                'pmra', 'pmdec', 
                                'classprob_dsc_combmod_star', 
                                'logg_gspphot',
                                'teff_gspphot'])

ztf = lsdb.read_hipscat("/data3/epyc/data3/hipscat/catalogs/ztf_axs/ztf_dr14", 
                       columns=['ra', 'dec', 'ps1_objid', 
                               'nobs_g', 'nobs_r', 'nobs_i', 
                               'Npix'])

#sources load takes a minute, since it creates a healpix alignment on load
ztf_sources = lsdb.read_hipscat("/data3/epyc/data3/hipscat/catalogs/ztf_axs/ztf_source",
                        columns=['index', 'ps1_objid',
                                       'ra', 'dec', 
                                       'catflags', 
                                       'fieldID', 
                                       'mjd', 'band', 'mag', 'magerr', 'Npix'])

In [12]:
# crossmatch ZTF + Gaia
_all_sky = gaia.crossmatch(ztf).query(
    "nobs_g_ztf_dr14 > 50 and nobs_r_ztf_dr14 > 50 and \
    parallax_gaia > 0 and parallax_over_error_gaia > 5 and \
    teff_gspphot_gaia > 5380 and teff_gspphot_gaia < 7220 and logg_gspphot_gaia > 4.5 \
    and logg_gspphot_gaia < 4.72 and classprob_dsc_combmod_star_gaia > 0.5").compute()


In [13]:
# catalog object
_sample_hips = lsdb.from_dataframe(
    _all_sky, 
    lowest_order=5, 
    highest_order=8, 
    set_hipscat_index=False, 
    ra_column="ra_gaia", 
    dec_column="dec_gaia", 
    threshold=1e10
)

# don't compute table...
ss = _sample_hips.join(
    ztf_sources, left_on="ps1_objid_ztf_dr14", right_on="ps1_objid"
)

In [14]:
print (f"Number of unique sources: {len(_all_sky.index.unique())}")

Number of unique sources: 4833834


In [15]:
# select only good catalog detections
good_detections_table = ss.query("catflags==0")

## TAPE

In [16]:
import dask.dataframe as dd
from tape import Ensemble, ColumnMapper

In [17]:
# Initialize an Ensemble
ens = Ensemble(client=client.client)
ens.client_info()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:43220/status,
Dashboard: http://127.0.0.1:43220/status,Workers: 12
Total threads: 12,Total memory: 0.98 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41765,Workers: 12
Dashboard: http://127.0.0.1:43220/status,Total threads: 12
Started: 14 minutes ago,Total memory: 0.98 TiB
Comm: tcp://127.0.0.1:36267,Total threads: 1
Dashboard: http://127.0.0.1:40468/status,Memory: 83.97 GiB
Nanny: tcp://127.0.0.1:34103,


In [18]:
# ColumnMapper Establishes which table columns map to timeseries quantities
colmap = ColumnMapper(
        id_col='_hipscat_index',
        time_col='mjd',
        flux_col='mag',
        err_col='magerr',
        band_col='band',
      )

# We can read from parquet
ens.from_dask_dataframe(
    source_frame=good_detections_table._ddf,
    object_frame=_sample_hips._ddf,
    column_mapper=colmap,
    sync_tables=False, # Avoid doing an initial sync
    sorted=True, # If the input data is already sorted by the chosen index
    sort=False,
)

In [19]:
pth = "/astro/users/atzanida/datalrg/datalrg2"

In [20]:
# store good detections... all sky?!
good_detections_table._ddf.to_parquet(f"{pth}/all_sky_fg_good_detections_table.parquet")

2023-12-07 20:44:05,693 - distributed.worker - WARNING - Compute Failed
Key:       ('to-parquet-94bce7ed5ab70db6bfe6e51bfff96465', 285589)
Function:  subgraph_callable-5389f9d5-1cf4-421c-a91c-f4d9f23a
args:      ((285589,), 'query-859f6617333cc8a8e6da7e6117085764', 'catflags==0', 'from-delayed-f9094cc579461a9fc208ab9df9f9f1e9',                          ra_gaia  dec_gaia  ...    magerr    Npix
_hipscat_index                              ...                  
13538452531557957633  288.948911 -26.00937  ...  0.012339  769571
13538452531557957633  288.948911 -26.00937  ...  0.012384  769571
13538452531557957633  288.948911 -26.00937  ...  0.012306  769571
13538452531557957633  288.948911 -26.00937  ...  0.012299  769571
13538452531557957633  288.948911 -26.00937  ...  0.012298  769571
...                          ...       ...  ...       ...     ...
13538452531557957633  288.948911 -26.00937  ...  0.019753  769571
13538452531557957633  288.948911 -26.00937  ...  0.019923  769571
1353845253

ValueError: Failed to convert partition to expected pyarrow schema:
    `ArrowInvalid('Integer value 13538452531557957633 not in range: 0 to 9223372036854775807', 'Conversion failed for column _hipscat_index with type uint64')`

Expected partition schema:
    ra_gaia: double
    dec_gaia: double
    bp_rp_gaia: double
    parallax_gaia: double
    parallax_over_error_gaia: double
    phot_g_mean_mag_gaia: double
    pmra_gaia: double
    pmdec_gaia: double
    classprob_dsc_combmod_star_gaia: double
    logg_gspphot_gaia: double
    teff_gspphot_gaia: double
    ra_ztf_dr14: double
    dec_ztf_dr14: double
    ps1_objid_ztf_dr14: int64
    nobs_g_ztf_dr14: int32
    nobs_r_ztf_dr14: int32
    nobs_i_ztf_dr14: int32
    Npix_ztf_dr14: int32
    _DIST: double
    index: int64
    ps1_objid: int64
    ra: double
    dec: double
    catflags: int16
    fieldID: int16
    mjd: double
    band: string
    mag: float
    magerr: float
    Npix: int32
    _hipscat_index: int64

Received partit

In [ ]:
# worked! 